In [ ]:
import cv2
import glob
import os
import sys
import numpy as np

from random import randint
import matplotlib.pyplot as plt

import socket
import json

from time import time
from cav.objects import Object, BoundingBox, ObjectType
from cav.parameters import Parameters

from cav.visualization import Map, plotBoxes, bsmImg

%matplotlib inline 

### BSM Server configuration

In [ ]:
PUSH_BSM = True # If true, bsms are being pushed to the server

HOST = '128.8.215.213'   # The server's hostname or IP address
PORT = 65432        # The port used by the server
DATA_BUFF = 4096

### Create object detector

In [ ]:
from cav.detection import ObjectDetector
import cv2
MODEL_PATH = '../../old_20190521/faster_rcnn_inception_v2/fine_tuned_model/frozen_inference_graph.pb'
od = ObjectDetector(MODEL_PATH)

### Create params and map objects

In [ ]:
params = Parameters()
params.generateParameters('./config/params.json')
mymap = Map('./images/CATT_satelite.jpg', './config/icons.json', params)
plt.imshow(mymap.getMap(), interpolation='nearest')
plt.show()

In [ ]:
def sendBsm(s, bsm):
    data = {
        'mode' : 'push',
        'msg' : bsm
    }
    
    msg = json.dumps(data)
    msg = str.encode(msg)
    s.sendall(msg)
    data = s.recv(1024)  
    return data    

In [ ]:

VIDEO_X = 240
VIDEO_Y = 320
FRAMES_SEC = 6

MAX_BOXES_TO_DRAW = 20
MIN_SCORE_THRESH = 0.5
IOU_COMMON_THRESHOLD = 0.50
NOT_DETECTED_TRHESHOLD = 1

ADD_BBOXES = False
ADD_MAP = False
BSM_X = 0

if ADD_MAP:
    FINAL_X = VIDEO_X + mymap.getMap().shape[1] + BSM_X
else:
    FINAL_X = VIDEO_X + BSM_X

FINAL_Y = VIDEO_Y 

#Set video output and img folder output (both can be None)
VIDEO_OUTPUT = None#'./stream_test.avi'
IMG_FOLDER_OUTPUT = None#'./tmp'


cap = cv2.VideoCapture('rtmp://170.93.143.157/rtplive/c5ff65d2008900a1004f823633235daa')
fourcc = cv2.VideoWriter_fourcc(*'XVID')

if VIDEO_OUTPUT is not None:
    out = cv2.VideoWriter(VIDEO_OUTPUT,fourcc, FRAMES_SEC, (int(FINAL_X), int(FINAL_Y)))

objects = []


time_total = time()
last_detect_time = 0
i = 0
test_nr_detections = 0

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    while cap.isOpened():
        sys.stdout.write("Processing Frame {}, Detections run {} times       \r".format(i, test_nr_detections))
        ret, image = cap.read()

        if image is None:
            print ("Warning! Image is none for frame {}".format(i))
            continue

        if len(objects) > 0:
            t = time()
            for obj in objects:
                obj.updateTracker(image, t) 

        
        if time() - last_detect_time > 2: # every 2 seconds
            boxes, scores, classes = od.detect(image)
            test_nr_detections += 1
            if len(boxes) > 0:
                last_detect_time  = time() # update only if something was detected
                if len(objects) > 0: ### Update bounding boxes
                    for obj in objects:
                        ious = obj.computeIoU(boxes)
                        argmax = np.argmax(ious)
                        if ious[argmax] > IOU_COMMON_THRESHOLD:
                            obj.updateBoundingBox(boxes[argmax])
                            boxes.pop(argmax)
                            scores = np.delete(scores, argmax)
                            classes = np.delete(classes, argmax)
                            if len(boxes) == 0:
                                break
                        else:
                            obj.notDetectedCounter = obj.notDetectedCounter + 1 

                for box, objclass in zip(boxes, classes): ### Create new Objects
                    obj = Object(ObjectType(objclass))
                    obj.createTracker(image, box)
                    obj.color = (randint(0, 255), randint(0, 255), randint(0, 255))
                    objects.append(obj)


        for obj in objects:
            if obj.notDetectedCounter >= NOT_DETECTED_TRHESHOLD:
                objects.remove(obj)
                del obj


        # Add bounding boxes
        
        
        
        if ADD_BBOXES:
            boxes = [obj.bboxes[-1] for obj in objects]

            if len(boxes) > 0:
                colors = [obj.color for obj in objects]
                vid = plotBoxes(image, boxes, colors=colors)
                if ADD_MAP:
                    mapimg = mymap.addObjects(objects)        
            else:
                vid = image.copy()
                if ADD_MAP:
                    mapimg = mymap.getMap()
        else:
            vid = image.copy()
            
        # Adding map
        if ADD_MAP:
            final_image = np.zeros((FINAL_Y, FINAL_X,3),dtype=np.uint8)

            final_image[:VIDEO_Y, :VIDEO_X, :] = vid
            final_image[:mapimg.shape[0], VIDEO_X:VIDEO_X + mapimg.shape[1], :] = mapimg  
        else:
            final_image = vid

        # Adding the BSM info on the image
        if BSM_X > 0:
            if len(objects) > 0:
                obj = objects[0]
                color = obj.color
                bsmimg = bsmImg(obj.getBsm(), framecolor=color)
                final_image[:bsmimg.shape[0], VIDEO_X + mapimg.shape[1]:, :] =  bsmimg

            if len(objects) > 1:
                obj = objects[1]
                color = obj.color
                bsmimg = bsmImg(obj.getBsm(), framecolor=color)
                final_image[bsmimg.shape[0]+10:2*bsmimg.shape[0]+10, VIDEO_X + mapimg.shape[1]:, :] =  bsmimg

        if VIDEO_OUTPUT is not None:
            out.write(final_image)

        if IMG_FOLDER_OUTPUT is not None:
            pass
            cv2.imwrite('{}/im_{}.jpg'.format(IMG_FOLDER_OUTPUT, str(i).zfill(4)), final_image)
        i = i+1
        if i > 60 * FRAMES_SEC:
        #if i > 3 * FRAMES_SEC:
            break

        if PUSH_BSM:
            bsm_list = []
            for obj in objects:
                bsm_list.append(obj.getBsm(retDic = False, params=params, roundValues = True, includeNone = False))
                
            data = sendBsm(s, json.dumps(bsm_list))
            #print ("Response: {}\n".format(data))
    
cap.release()

time_total = time() - time_total
print ('{} frames processed in {:.1f} seconds ({:.1f} frames / second)'
      .format(i, time_total, i / time_total))
